# Practical 8: Physical and Hybrid Input-Output Tables

Objectives
- Understand the structure of Hybrid Input-Output Tables (HIOT)
- Analyze HIOT data

Download the the MR_HIOT data from: https://zenodo.org/record/7244919 (n.b. ignore files that start with MR_HSUT or MR_HUSE )

In [1]:
# Import packages
import pandas as pd
import numpy as np

### Exercise 1: Understand data availability in HIOT

Import the following data

- MR_HIOT_2011_v3_3_18_xx_principal_production.csv indicates the principal production of the productive activities;
- MR_HIOT_2011_v3_3_18_xx_by_product_technology.csv indicates the matrix of the uses where by-products are included with a negative sign;
- MR_HIOT_2011_v3_3_18_xx_FD.csv includes the consumption of final consumers
- MR_HIOT_2011_v3_3_18_xx_stock_to_waste.csv includes the demand of waste treatment services to treat the stock reduction
- MR_HIOT_2011_v3_3_18_xx_extensions.csv includes the extensions as reported in the HSUTs. Very minor changes are applied


In this practical you will only use a few of the imported datasets, this exercise is to help you understand the variety of the data. If you so wish, you may also perform additional analysis with the imported datasets 

1.1 Inter-industry matrix

In [2]:
Z = pd.read_csv("data/MR_HIOT_2011_v3_3_18_by_product_technology.csv", index_col=[0,1,2,3,4], header=[0,1,2,3])

1.2 Final demand

In [3]:
Y  = pd.read_csv("data/MR_HIOT_2011_v3_3_18_FD.csv", index_col=[0,1,2,3,4], header=[0,1,2,3])

1.3 Extensions

In [4]:
extensions = pd.ExcelFile("data/MR_HIOT_2011_v3_3_18_extensions.xlsx")

In [5]:
extensions.sheet_names

['intro',
 'resource_act',
 'resource_FD',
 'Land_act',
 'Land_FD',
 'Emiss_act',
 'Emiss_FD',
 'Emis_unreg_w_act',
 'Emis_unreg_w_FD',
 'waste_sup_act',
 'waste_sup_FD',
 'waste_use_act',
 'waste_use_FD',
 'pack_sup_waste_act',
 'pack_sup_waste_fd',
 'pack_use_waste_act',
 'pack_use_waste_fd',
 'mach_sup_waste_act',
 'mach_sup_waste_fd',
 'mach_use_waste_act',
 'mach_use_waste_fd',
 'waste_from_stocks',
 'stock_addition_act',
 'stock_addition_fd',
 'crop_res_act',
 'crop_res_FD',
 'VA_act']

1.4 Resource extraction

In [6]:
# Resource extraction matrix of Z
RE = extensions.parse(sheet_name="resource_act", index_col=[0,1], header=[0,1,2,3]) 
# Resource extraction matrix of Y
RE_FD = extensions.parse(sheet_name="resource_FD", index_col=[0,1], header=[0,1,2,3]) 

1.5 Waste supply and use accounts

In [7]:
# Waste supply matrix of Z
WS = extensions.parse(sheet_name="waste_sup_act", index_col=[0,1], header=[0,1,2,3]) 
# Waste supply matrix of Y
WS_FD = extensions.parse(sheet_name="waste_sup_FD", index_col=[0,1], header=[0,1,2,3]) 

# Waste use matrix of Z
WU = extensions.parse(sheet_name="waste_use_act", index_col=[0,1], header=[0,1,2,3]) 
# Waste use matrix of Y => This is all 0's so it can also be ignored
WU_FD = extensions.parse(sheet_name="waste_use_FD", index_col=[0,1], header=[0,1,2,3]) 

1.6 Stock additions

In [8]:
# Stock addition matrix of Z
SA = extensions.parse(sheet_name="stock_addition_act", index_col=[0,1], header=[0,1,2,3]) 
# Stock addition matrix of Y
SA_FD = extensions.parse(sheet_name="stock_addition_fd", index_col=[0,1], header=[0,1,2,3]) 

1.7 Stock depletion matrix

In [9]:
# Stock depletion matrix
SD = extensions.parse(sheet_name="waste_from_stocks", index_col=[0,1], header=[0,1,2,3])

1.8 Emissions

In [10]:
EM = extensions.parse(sheet_name="Emiss_act", index_col=[0,1,2], header=[0,1,2,3])
EM_FD = extensions.parse(sheet_name="Emiss_FD", index_col=[0,1,2], header=[0,1,2,3])

## Exercise 2: Calculate the global circularity gap

Circularity gap in absolute terms

$ CG = W_{sup} + S_{dep} - W_{rec}$

Where:
- $CG$ is the circularity gap
- $W_{sup}$ is the global waste supply
- $S_{dep}$ is the global stock depletation 
- $W_{rec}$ is the global waste use/recovery 


Circularity gap index

$ CGI = \frac{CG}{W_{sup} + S_{dep}} \times 100$



Methods from:
 
Aguilar-Hernandez, G. A., Sigüenza-Sanchez, C. P., Donati, F., Merciai, S., Schmidt, J., Rodrigues, J. F., & Tukker, A. (2019). The circularity gap of nations: A multiregional analysis of waste generation, recovery, and stock depletion in 2011. Resources, Conservation and Recycling, 151, 104452.

### 2.1: Calculate the global total waste generation

In [11]:
WS_tot = WS.sum().sum() + WS_FD.sum().sum()
WS_tot

14577617331.961227

### 2.2: Calculate global total waste use (i.e., recovery)

In [12]:
WU_tot = WU.sum().sum() + WU_FD.sum().sum()
WU_tot

16712487923.123253

### 2.3: Calculate global total stock degradation

In [13]:
SD_tot = SD.sum().sum()

SD_tot

4736719890.900244

### 2.4: Circularity gap calculations


In [14]:
# CIRCULARITY GAP CALCULATION
circularity_gap = WS_tot + SD_tot - WU_tot

print(f"Global Circularity Gap: {round(circularity_gap/(WS_tot + SD_tot)*100, 1)} %")

Global Circularity Gap: 13.5 %


## Exercise 3: Sankey Diagram of global material flows

In this exercise you will use precalculate global material flows obtained from an earlier version of exiobase MRHIOT (v3.3.15)

Please, ensure that floweaver and ipysankeywidget are installed in your environment

In [15]:
import floweaver as fw
from ipysankeywidget import SankeyWidget

We want to create a Sankey diagram of the global material inputs and outputs. Figure 1 shows a diagram of the inputs and outputs of an economy. 

<image src="figures\figure1.jpg" width="60%"></image>


Figure 1. System definition of the input-output material flows of a country. Solid blocks indicate economic activities of: 
- I&C = Intermediate sectors and final demand; 
- T = waste treatment activities; 
- RoW = Rest of the world.  Solid circles indicate resource stocks of:  
    - N = Natural resources; 
    - S = Material in-use stocks; 
    - DPO = Domestic processed output. 
- Solid and dashed lines indicate flows of: 
    - m = imports; 
    - r = resource extraction domestically; 
    - w_rec = waste recovery; 
    - e = exports; 
    - s_add  = stock additions; 
    - s_dep = stock depletion, 
    - w_sup = waste generation; 
    - b_(I&C) = dissipative emissions, others combustion and biomass residues from intermediate activities and final demand; 
    - and b_T   = dissipative emissions and others combustion and biomass residues from waste treatment.


The input and outputs are balanced as:

$ m + r + w_{rec}= e + b_{I\&C} + w_{sup} + S_{add} $

When considering the global inputs and outputs, imports and exports are zero. Thus, for global inputs and outputs:

$ r + w_{rec} = b_{I\&C} + w_{sup} + S_{add} $



In [16]:
# Use this data for you calculations
material_flows = {'Gigatonnes (Gt)': {'Material extraction': 73.475,
                                      'Emissions': 40.529,
                                      'Waste': 6.438,
                                      'Inflows-to-Stocks': 29.624,
                                      'Outflows-from-Stock': 2.268,
                                      'Waste-from-treatment': 5.590,
                                      'Waste-recovery': 3.116}}

material_flows = pd.Series(material_flows['Gigatonnes (Gt)'])
material_flows

Material extraction     73.475
Emissions               40.529
Waste                    6.438
Inflows-to-Stocks       29.624
Outflows-from-Stock      2.268
Waste-from-treatment     5.590
Waste-recovery           3.116
dtype: float64

### 3.1 Based on Figure 1, create a new dataframe with the Floweaver structure. 

Floweaver requires a pd.dataframe with four columns

| Source |	Target  | Type  | Value |
|--------|----------|-------|-------|
| [Initial node]    | [Final node]  |	[Name of flow]  | [Value of flow]| 

For example, in figure 1, material extraction (N) is an initial node, demand (I&C) is a final node, with flow type called “material”, and a value of 73.4 Gt (which is allocated in material_flows.iloc[0,0]]). 



In [17]:
flows = pd.DataFrame([['N', 'I&C', 'all_mat', material_flows.iloc[0]],
                      ['I&C', 'E', 'all_mat', material_flows.iloc[1]],
                      ['I&C', 'T', 'all_mat', material_flows.iloc[2]],
                      ['I&C', 'S', 'all_mat', material_flows.iloc[3]],
                      ['S', 'T', 'fossil', material_flows.iloc[4]],
                      ['T', 'W', 'all_mat', material_flows.iloc[5]],
                      ['T', 'I&C', 'all_mat', material_flows.iloc[6]]]) # Dataframe in structure required by Floweaver
flows.columns = ['source', 'target', 'type', 'value']

flows

,source,target,type,value
0,N,I&C,all_mat,73.475
1,I&C,E,all_mat,40.529
2,I&C,T,all_mat,6.438
3,I&C,S,all_mat,29.624
4,S,T,fossil,2.268
5,T,W,all_mat,5.590
6,T,I&C,all_mat,3.116


### 3.2 Create Floweaver settings (nodes, ordering, and bundle)

In [18]:
# Nodes = Group of processes from the underlying dataset
nodes = {'N': fw.ProcessGroup(['N']),
        'I&C': fw.ProcessGroup(['I&C']),
         'S': fw.ProcessGroup(['S']),
         'T': fw.ProcessGroup(['T']),
         'DPO': fw.ProcessGroup(['E', 'W']),
         'rec': fw.Waypoint(direction='L'), } 

# Ordering = The order of processes in the diagram
ordering = [[['N'], []],
           [['I&C'], ['rec']],
           [['S'], []],
           [['T'], []],
           [['DPO'], []]] 

# Bundle = a set of flows between two processes
bundles = [fw.Bundle('N', 'I&C'),
           fw.Bundle('I&C', 'T'),
           fw.Bundle('I&C', 'S'),
           fw.Bundle('I&C', 'DPO'),
           fw.Bundle('S', 'T'),
           fw.Bundle('T', 'DPO'),
           fw.Bundle('T', 'I&C'), ] 

### 3.3 Create Sankey diagram with the following code

In [19]:
sdd = fw.SankeyDefinition(nodes, bundles, ordering) # Connect all settings
size = dict(width=750, height=300)  # Adjust diagram size
fw.weave(sdd, flows, palette=['green']).to_widget(**size) # Display Sankey diagram

SankeyWidget(groups=[{'id': '__T_I&C_1', 'type': 'group', 'title': '', 'nodes': ['__T_I&C_1^*']}, {'id': '__I&…